In [1]:
!pip install names streamlit pandas textblob altair vaderSentiment ollama
# Import necessary libraries
import pandas as pd
from datetime import datetime
from textblob import TextBlob
import altair as alt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import requests
import random
import names
import ollama

# Load zodiac data
def load_zodiac_data():
    return pd.read_csv('zodiac_data.csv')

# Find zodiac sign
def find_zodiac(month, day):
    zodiac_signs = {
        'december': ('Sagittarius', 'Capricorn', 22),
        'january': ('Capricorn', 'Aquarius', 20),
        'february': ('Aquarius', 'Pisces', 19),
        'march': ('Pisces', 'Aries', 21),
        'april': ('Aries', 'Taurus', 20),
        'may': ('Taurus', 'Gemini', 21),
        'june': ('Gemini', 'Cancer', 21),
        'july': ('Cancer', 'Leo', 23),
        'august': ('Leo', 'Virgo', 23),
        'september': ('Virgo', 'Libra', 23),
        'october': ('Libra', 'Scorpio', 23),
        'november': ('Scorpio', 'Sagittarius', 22) 
    }
    sign1, sign2, split_day = zodiac_signs.get(month.lower(), (None, None, None))
    return sign1 if day < split_day else sign2

# Generate name using LLaMA 2
def generate_name(zodiac_sign, gender):
    prompt = f"Generate a unique {gender} name that embodies the qualities of the {zodiac_sign} zodiac sign."
    response = ollama.generate(model='llama2:7b', prompt=prompt)
    return response['response'].strip()

# Ollama integration (for generating zodiac descriptions)
def generate_with_ollama(prompt, model="llama2"):
    url = "http://localhost:11434/api/generate"
    data = {
        "model": model,
        "prompt": prompt
    }
    response = requests.post(url, json=data)
    if response.status_code == 200:
        return response.json()["response"]
    else:
        return f"Error: {response.status_code}"

# Sentiment Analysis Functions
def convert_to_df(sentiment):
    sentiment_dict = {'polarity': sentiment.polarity, 'subjectivity': sentiment.subjectivity}
    sentiment_df = pd.DataFrame(sentiment_dict.items(), columns=['metric', 'value'])
    return sentiment_df

def analyze_token_sentiment(docx):
    analyzer = SentimentIntensityAnalyzer()
    pos_list = []
    neg_list = []
    neu_list = []
    for word in docx.split():
        res = analyzer.polarity_scores(word)['compound']
        if res > 0.1:
            pos_list.append(word)
        elif res <= -0.1:
            neg_list.append(word)
        else:
            neu_list.append(word)
    result = {'positives': pos_list, 'negatives': neg_list, 'neutral': neu_list}
    return result

# Main function to run the app
def main():
    # Set up initial variables
    dob = None
    gender = None
    zodiac_sign = None
    name = None
    description = None
    sentiment = None
    zdf = None

    # User Input
    print("📅 Enter Your Date of Birth and Gender")
    dob_input = input("Date of Birth (YYYY-MM-DD): ")
    dob = datetime.strptime(dob_input, '%Y-%m-%d')
    gender = input("Gender (Male/Female/Non-binary): ")

    # Generate Zodiac Sign
    if input("Generate Zodiac Sign? (yes/no): ").lower() == 'yes':
        zodiac_sign = find_zodiac(dob.strftime("%B"), dob.day)
        print(f"🌟 Your Zodiac Sign: {zodiac_sign}")

    # Generate Name
    if zodiac_sign and input("Generate Name? (yes/no): ").lower() == 'yes':
        name = generate_name(zodiac_sign, gender)
        print(f"Generated Name: {name}")

    # Load Zodiac Data and Perform Sentiment Analysis
    if input("Perform Sentiment Analysis? (yes/no): ").lower() == 'yes':
        zodiac_df = load_zodiac_data()
        zdf = zodiac_df[zodiac_df['horoscope'] == zodiac_sign.title()]

        if not zdf.empty:
            description = zdf.iloc[0].description
            print(description)

            sentiment = TextBlob(description).sentiment
            print("Sentiment Analysis of Zodiac Description")
            result_df = convert_to_df(sentiment)
            print(result_df)

            if sentiment.polarity > 0:
                print("Sentiment: Positive 😊")
            elif sentiment.polarity < 0:
                print("Sentiment: Negative 😞")
            else:
                print("Sentiment: Neutral 😐")

            token_sentiments = analyze_token_sentiment(description)
            print("Positive words:", ", ".join(token_sentiments['positives']))
            print("Negative words:", ", ".join(token_sentiments['negatives']))
            print("Neutral words:", ", ".join(token_sentiments['neutral']))

if __name__ == '__main__':
    main()


📅 Enter Your Date of Birth and Gender


Date of Birth (YYYY-MM-DD):  2016-10-11
Gender (Male/Female/Non-binary):  Male
Generate Zodiac Sign? (yes/no):  yes


🌟 Your Zodiac Sign: Libra


Generate Name? (yes/no):  yes


Generated Name: Certainly! Based on the qualities associated with the Libra zodiac sign, I present to you the name:

Asher Malcolm Thorne

Breakdown of the name:

* Asher: derived from the Hebrew word for "happy," which aligns with Libra's association with balance and harmony.
* Malcolm: a strong, masculine name that represents stability and dependability, two qualities often associated with Libras.
* Thorne: a surname that evokes a sense of thorny complexity, mirroring Libra's reputation for being diplomatic and adaptable in challenging situations.

Together, these names create a unique and balanced identity that captures the essence of a Libra male: someone who is happy, stable, dependable, and skilled at navigating complex situations with grace and poise.


Perform Sentiment Analysis? (yes/no):  yes


The zodiac sign Libra is thrives when their needs of balance, justice, and stability are met. They are charming creatures that somehow always surround themselves with a sense of beauty and harmony. Admittedly, some of them can go to extremes searching for that harmony - which can make their situations unreasonable or unhealthy. Their ruling planet is Venus meaning that Libras are nurturing, caring, and they can make great defenders of the downtrodden. Sometimes, they can be shy if they find difficulties in coming out of their shell and letting their guard down. Despite their more introverted side they still love a good debate.
Sentiment Analysis of Zodiac Description
         metric     value
0      polarity  0.268056
1  subjectivity  0.617361
Sentiment: Positive 😊
Positive words: justice,, charming, beauty, harmony., harmony, nurturing,, caring,, great, love, good
Negative words: unhealthy., shy, difficulties
Neutral words: The, zodiac, sign, Libra, is, thrives, when, their, needs, of